<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Scénarios d'Exploration Statistique](https://github.com/wikistat/Exploration)

# [ACP](http://wikistat.fr/pdf/st-m-explo-acp.pdf) de données cubiques en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a> avec <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="Scikit-Learn"/></a>
##  Données de l'OCDE
### Résumé


## 1 Introduction
### 1.1 Objectif
L'objectif de cette section  est l'exploration de données socio-économiques plus complexes. La principale spécificité de ces données est de se présenter  sous la forme d'un cube de données ou tableau à trois entrées: le numéro de ligne, le numéro de variable et l'année d'observation de cette variable. Après une description classique, la mise en oeuvre de l'analyse en composantes principales avec python nécessite un effort particulier afin de produire les graphes adaptés à la structure particulière des données. Comme souvent, les graphes par défaut ne suffisent pas.

### 1.2 Les données
Les données sont issues de l'Observatoire de l'OCDE.  Pour chaque pays membre et pour chacune des années  1975, 1977, 1979, 1981, on connaît les valeurs prises par les  variables suivantes qui sont toutes des \emph{taux}~:
- Taux brut de natalité, 
- Taux de chômage, 
- Pourcentage d'actifs dans le secteur primaire, 
- Pourcentage d'actifs dans le secteur secondaire, 
- produit intérieur brut (par habitant), 
- Formation brute de capital fixe (par habitant), 
- Hausse des prix, 
- Recettes courantes  (par habitant), 
- Mortalité infantile, 
- Consommation de protéines animales  (par habitant), 
- Consommation d'énergie  (par habitant).

Elles sont disponibles dans le fichier: `ocdeR.dat`.

Les mêmes variables sont donc observées, sur les mêmes pays ou individus à quatre dates différentes. Plusieurs stratégies d'analyse sont possibles (tableau moyen, tableaux concaténés, meilleur compromis). La plus adaptée pour ces données est de considérer les observations des variables pour chacun des individus:  pays $\times$ années. 


In [ ]:
# Importaiton des principals librairies et 
# Affichage des graphiques dans le notebook
%matplotlib inline 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 2 Lecture des données

In [ ]:
ocde=pd.read_table("Data/ocdeR.dat",sep='\s+',index_col=0)
ocde.head()

## 3 Exploration
### 3.1 Statistiques élémentaires
Consulter rapidement ces résultats; Que dire à propos de la symétrie des distributions, de leur normalité, des valeurs atypiques.

In [ ]:
ocde.mean()

In [ ]:
ocde["CNRJ"].hist(bins=20)
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(ocde, alpha=0.2, figsize=(15, 15), diagonal='kde')
plt.show()

**Q** Que dire de la structure de corrélation ?

### 3.1 [Analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf)
Chaque pays étant observé 4 fois, la principale difficulté technique est de faire apparaître cette structure chronologique dans les graphique afin d'illustrer la dynamique économique de la période considérée.
#### Composantes principales

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
ocdeS=scale(ocde)
pca = PCA()
cpOcde = pca.fit_transform(ocdeS)

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.show()

In [ ]:
## distribution des composantes principales
plt.boxplot(cpOcde)
plt.show()

**Q** Combien de composantes retenir? Pourquoi?

#### Représentation des variables

In [ ]:
## coordonnées et représentation des variables
coord1=pca.components_[0]*np.sqrt(pca.explained_variance_[0])
coord2=pca.components_[1]*np.sqrt(pca.explained_variance_[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1)
for i, j, nom in zip(coord1,coord2, ocde.columns):
    plt.text(i, j, nom)
    plt.arrow(0,0,i,j,color='black')
plt.axis((-1.2,1.2,-1.2,1.2))
# cercle
c=plt.Circle((0,0), radius=1, color='gray', fill=False)
ax.add_patch(c)
plt.show()

**Q** Interprétation des axes 1, 2, 3?

#### Représentation basique des individus

In [ ]:
plt.figure(figsize=(10,6))
for i, j, nom in zip(cpOcde[:,0], cpOcde[:,1], ocde.index):
#    color = int(i/4)
    plt.text(i, j, nom ,color="blue")
plt.axis((-5,7,-4,4))  
plt.show()

#### Représentation adaptée à des données temporelles

In [ ]:
import matplotlib.patheffects as PathEffects

comp_0 = 0
comp_1 = 1

cmap = plt.get_cmap("tab20")

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(1,1,1)
for i,k in enumerate(np.arange(0,cpOcde.shape[0],4)):
    country =ocde.index[k]
    xs = cpOcde[k:k+4,comp_0]
    ys = cpOcde[k:k+4, comp_1]
    ax.plot(xs,ys, color=cmap(i), marker=".", markersize=15)
    txt = ax.text(xs[-4], ys[-4], country, horizontalalignment="left", verticalalignment="top",
            color=cmap(i), fontweight="bold", fontsize=15)
    # Add black line around text
    #txt.set_path_effects([PathEffects.withStroke(linewidth=1, foreground='black')])
    ax.set_xlabel("PC%d" %comp_0, fontsize=20)
    ax.set_ylabel("PC%d" %comp_1, fontsize=20)
plt.tight_layout()
plt.show()